#Sistema de recomendação

Content based filtering and collaborative filtering

#Objetivo

Criar um sistema de recomendação de filmes 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
filmes = pd.read_csv('movies.csv')

filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
filmes.columns = ['filmeId','titulo','generos']

In [4]:
notas = pd.read_csv('ratings.csv')
notas.columns = ['usuarioId','filmeId','nota','momento']
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#Primeira tentativa de recomendação

In [5]:
total_votos = notas['filmeId'].value_counts()
total_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: filmeId, dtype: int64

In [6]:
filmes = filmes.set_index('filmeId')

In [7]:
filmes.loc[318]

titulo     Shawshank Redemption, The (1994)
generos                         Crime|Drama
Name: 318, dtype: object

Apesar de não saber nada sobre o usuário, posso recomendar os filmes baseados em quão votado ele foi, pois se ele foi muito votado, significa dizer que ele foi muito visto, ou seja, muitas pessoas gostam desse tipo de filme.

In [8]:
filmes['total_votos'] = total_votos
filmes.head()

,titulo,generos,total_votos
filmeId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [9]:
filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos
filmeId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


In [10]:
notas_medias = notas.groupby(by='filmeId').mean()['nota']
notas_medias.head()

filmeId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: nota, dtype: float64

In [11]:
filmes['nota_media'] = notas_medias
filmes.sort_values('total_votos', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


Este é o primeiro sistema de recomendação, mais básico, no qual sugere um filme baseado na quantidade total de votos e apresenta a nota média do filme levando em consideração todos os usuários que votaram nesse filme.

Não necessariamente o filme mais votado, é o filme que as pessoas mais gostaram. Logo, iremos fazer a segunda heurística.

# Segunda heurística

In [12]:
filmes.sort_values('nota_media', ascending=False).head(10)

,titulo,generos,total_votos,nota_media
filmeId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


O problema de recomendação é mais complexo por mais que uma apenas ordenação. Para isso, precisamos aumentar o número total de votos mínimos.

In [13]:
filmes_50votos = filmes.query('total_votos >=50').sort_values('nota_media', ascending=False)
filmes_50votos.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


#Distância entre usuários do sistema

In [14]:
def notas_do_usuario(usuario):
  notas_usuario = notas.query('usuarioId==%d' % usuario)
  notas_usuario = notas_usuario[['filmeId','nota']].set_index('filmeId')
  return notas_usuario

In [15]:
usuario1 = notas_do_usuario(1)
usuario4 = notas_do_usuario(4)

In [16]:
diferencas = usuario1.join(usuario4, lsuffix='_esq', rsuffix='_dir').dropna()

In [17]:
#distancia de dois usuários
np.linalg.norm(diferencas['nota_esq'] - diferencas['nota_dir'])

11.135528725660043

In [18]:
def distancia_usuarios(usuario_id1, usuario_id2):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  diferencas = notas1.join(notas2, lsuffix='_esq', rsuffix='_dir').dropna()
  distancia = np.linalg.norm(diferencas['nota_esq'] - diferencas['nota_dir'])
  return [usuario_id1, usuario_id2, distancia]

In [19]:
distancia_usuarios(1, 4)

[1, 4, 11.135528725660043]

In [20]:
quant_usuarios = len(notas.usuarioId.unique())
print('temos %d usuarios' % quant_usuarios)

temos 610 usuarios


In [21]:
primeiro_id = 1
distancias = []
for usuario_id in notas.usuarioId.unique():
  informacoes = distancia_usuarios(primeiro_id, usuario_id)
  distancias.append(informacoes)

In [22]:
distancias[:5]

[[1, 1, 0.0],
 [1, 2, 1.4142135623730951],
 [1, 3, 8.200609733428363],
 [1, 4, 11.135528725660043],
 [1, 5, 3.7416573867739413]]

In [23]:
def distancia_todos(primeiro_id):
  distancias = []
  for usuario_id in notas.usuarioId.unique():
    informacoes = distancia_usuarios(primeiro_id, usuario_id)
    distancias.append(informacoes)
    distancia = pd.DataFrame(distancias, columns = ['voce','outra_pessoa','distancia'])
  return distancia

distancia_todos(1).head()

,voce,outra_pessoa,distancia
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657


# Usuários sem filmes em comum são colocados bem distante um do outro

In [24]:
def distancia_usuarios(usuario_id1, usuario_id2, minimo = 5):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  diferencas = notas1.join(notas2, lsuffix='_esq', rsuffix='_dir').dropna()

  if (len(diferencas) < minimo):
    return [usuario_id1, usuario_id2, 100000]
    
  distancia = np.linalg.norm(diferencas['nota_esq'] - diferencas['nota_dir'])
  return [usuario_id1, usuario_id2, distancia]

In [25]:
distancia_todos(5).head()

,voce,outra_pessoa,distancia
0,5,1,3.741657
1,5,2,100000.000000
2,5,3,100000.000000
3,5,4,6.324555
4,5,5,0.000000


In [26]:
def usuarios_similares(primeiro_id):
  distancias = distancia_todos(primeiro_id)
  distancias = distancias.sort_values('distancia')
  return distancias

In [27]:
usuarios_similares(77)

,voce,outra_pessoa,distancia
0,77,1,0.0
266,77,267,0.0
76,77,77,0.0
38,77,39,1.0
493,77,494,1.0
...,...,...,...
376,77,377,100000.0
161,77,162,100000.0
374,77,375,100000.0
383,77,384,100000.0


verificando a similaridade tão forte

In [28]:
notas_do_usuario(267).join(notas_do_usuario(77), lsuffix='1', rsuffix='77').dropna()

,nota1,nota77
filmeId,,
260,5.0,5.0
1196,5.0,5.0
1198,5.0,5.0
1210,5.0,5.0
2571,5.0,5.0
3578,5.0,5.0
3996,5.0,5.0


Removendo a distância do primeiro_id com ele mesmo

In [29]:
def usuarios_similares(primeiro_id):
  distancias = distancia_todos(primeiro_id)
  distancias = distancias.sort_values('distancia')
  distancias = distancias.set_index('outra_pessoa').drop(primeiro_id)
  return distancias

In [30]:
usuarios_similares(8)

,voce,distancia
outra_pessoa,,
338,8,0.707107
343,8,1.000000
246,8,1.224745
215,8,1.322876
543,8,1.414214
...,...,...
406,8,100000.000000
252,8,100000.000000
253,8,100000.000000


In [31]:
def usuarios_similares(primeiro_id, n = None):
  distancias = distancia_todos(primeiro_id, n = n)
  distancias = distancias.sort_values('distancia')
  distancias = distancias.set_index('outra_pessoa').drop(primeiro_id)
  return distancias

In [32]:
def distancia_todos(primeiro_id, n = None):
  distancias = []
  todos_usuarios = notas.usuarioId.unique()
  if n:
    todos_usuarios = todos_usuarios[:n]
  for usuario_id in todos_usuarios:
    informacoes = distancia_usuarios(primeiro_id, usuario_id)
    distancias.append(informacoes)
    distancia = pd.DataFrame(distancias, columns = ['voce','outra_pessoa','distancia'])
  return distancia

In [33]:
usuarios_similares(1, n = 50)

,voce,distancia
outra_pessoa,,
49,1,1.000000
9,1,1.000000
13,1,1.414214
25,1,1.414214
30,1,1.802776
35,1,2.236068
26,1,2.236068
46,1,3.316625
8,1,3.741657


In [34]:
def distancia_usuarios(usuario_id1, usuario_id2, minimo = 5):
  notas1 = notas_do_usuario(usuario_id1)
  notas2 = notas_do_usuario(usuario_id2)
  diferencas = notas1.join(notas2, lsuffix='_esq', rsuffix='_dir').dropna()

  if (len(diferencas) < minimo):
    return None
    
  distancia = np.linalg.norm(diferencas['nota_esq'] - diferencas['nota_dir'])
  return [usuario_id1, usuario_id2, distancia]

In [35]:
def distancia_todos(primeiro_id, n = None):
  distancias = []
  todos_usuarios = notas.usuarioId.unique()
  if n:
    todos_usuarios = todos_usuarios[:n]
  for usuario_id in todos_usuarios:
    informacoes = distancia_usuarios(primeiro_id, usuario_id)
    distancias.append(informacoes)
    distancias = list(filter(None, distancias)) #filter arranca todos os Nones
    distancia = pd.DataFrame(distancias, columns = ['voce','outra_pessoa','distancia'])
  return distancia

In [36]:
usuarios_similares(1, n=50)

,voce,distancia
outra_pessoa,,
49,1,1.000000
9,1,1.000000
25,1,1.414214
13,1,1.414214
30,1,1.802776
35,1,2.236068
26,1,2.236068
46,1,3.316625
8,1,3.741657


In [37]:
voce = 1

def sugere_para(voce, n = None):
  similares = usuarios_similares(voce, n = n) 
  similar = similares.iloc[0].name
  notas_voce = notas_do_usuario(voce)
  filmes_q_vc_viu = notas_voce.index

  notas_similar = notas_do_usuario(similar)
  notas_similar = notas_similar.drop(filmes_q_vc_viu, errors='ignore')
  recomendacoes = notas_similar.sort_values('nota', ascending=False)
  return recomendacoes.join(filmes)

In [38]:
sugere_para(1, n = 50).head()

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
1200,4.5,Aliens (1986),Action|Adventure|Horror|Sci-Fi,126.0,3.964286
4022,4.5,Cast Away (2000),Drama,100.0,3.700000
47099,4.5,"Pursuit of Happyness, The (2006)",Drama,46.0,3.793478
79132,4.5,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,143.0,4.066434
109487,4.5,Interstellar (2014),Sci-Fi|IMAX,73.0,3.993151


O algortimo já é mais complexo que o primeiro, contudo, ele sugere os 50 primeiros filmes baseados em um usuário no banco de dados que tem o gosto mais similar com o usuário que está fazendo a busca (no caso, usuário 1). Mas poderíamos levar em consideração outros usuários similares, não só o exatamente mais próximo...

#Sugerindo baseado em vários usuários

In [39]:
def usuarios_similares(primeiro_id, n_mais_proximos=10, n = None):
  distancias = distancia_todos(primeiro_id, n = n)
  distancias = distancias.sort_values('distancia')
  distancias = distancias.set_index('outra_pessoa').drop(primeiro_id)
  return distancias.head(n_mais_proximos)

In [40]:
usuarios_similares(1, n_mais_proximos=2, n=300)

,voce,distancia
outra_pessoa,,
77,1,0.0
258,1,1.0


In [41]:
voce=1
def sugere_para(voce, n_mais_proximos=10, n=None):
  notas_voce = notas_do_usuario(voce)
  filmes_q_vc_viu = notas_voce.index
  
  similares = usuarios_similares(voce, n_mais_proximos = n_mais_proximos, n = n)   
  usuarios_parecidos = similares.index
  notas_similares = notas.set_index('usuarioId').loc[usuarios_parecidos]
  recomendacoes = notas_similares.groupby('filmeId').mean()[['nota']]
  recomendacoes = recomendacoes.sort_values('nota', ascending=False)
  return recomendacoes.join(filmes)


In [42]:
sugere_para(1, n_mais_proximos = 2, n=300).head()

,nota,titulo,generos,total_votos,nota_media
filmeId,,,,,
260,5.0,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
8961,5.0,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,125.0,3.836000
5378,5.0,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX,92.0,3.157609
5816,5.0,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy,102.0,3.598039
5952,5.0,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,188.0,4.021277


esse modelo funcionaria quando conhecemos os usuários (exigiria um cadastro). Para fazer o deploy, vou optar por uma outra abordagem que sugestiona um filme de acordo com uma lista de filmes que o usuário escolherá, de acordo com seu próprio gosto.

#Collaborative Filtering

In [43]:
notas.head()

,usuarioId,filmeId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [44]:
filmes_50votos.head()

,titulo,generos,total_votos,nota_media
filmeId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041


In [45]:
notas_filmes = filmes_50votos.merge(notas, on='filmeId')
notas_filmes = notas_filmes[['filmeId','titulo','generos','usuarioId','nota']]
notas_filmes.head()

,filmeId,titulo,generos,usuarioId,nota
0,318,"Shawshank Redemption, The (1994)",Crime|Drama,2,3.0
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,5,3.0
2,318,"Shawshank Redemption, The (1994)",Crime|Drama,6,5.0
3,318,"Shawshank Redemption, The (1994)",Crime|Drama,8,5.0
4,318,"Shawshank Redemption, The (1994)",Crime|Drama,11,4.0


In [46]:
pt = notas_filmes.pivot_table(index = 'titulo', columns = 'usuarioId',
                       values= 'nota').fillna(0)
pt.head()

usuarioId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
titulo,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [47]:
from sklearn.metrics.pairwise import cosine_similarity

similaridade = cosine_similarity(pt)

def recomendador(nome_filme):
    index = np.where(pt.index == nome_filme)[0][0]
    filmes_similares = sorted(enumerate(similaridade[index]),key= lambda x: x[1],
                              reverse =True)[1:6]
    
    for i in filmes_similares:
        print(pt.index[i[0]])

In [48]:
recomendador('2001: A Space Odyssey (1968)')

Blade Runner (1982)
Alien (1979)
Apocalypse Now (1979)
Aliens (1986)
Clockwork Orange, A (1971)


In [49]:
import pickle

pickle.dump(recomendador, open(r'C:\Users\julio\OneDrive\Área de Trabalho\deploy_ml_project\deploy\model.pkl','wb'))

In [59]:
pickle.dump(filmes_50votos, open(r'C:\Users\julio\OneDrive\Área de Trabalho\deploy_ml_project\deploy\dataset.pkl','wb'))

In [51]:
pickle.dump(pt, open(r'C:\Users\julio\OneDrive\Área de Trabalho\deploy_ml_project\deploy\pivot_table.pkl','wb'))

In [61]:
pickle.dump(filmes, open(r'C:\Users\julio\OneDrive\Área de Trabalho\deploy_ml_project\deploy\filmes.pkl','wb'))

In [55]:
pickle.dump(similaridade, open(r'C:\Users\julio\OneDrive\Área de Trabalho\deploy_ml_project\deploy\similaridade.pkl','wb'))